# Astropy Units Integration
Author: Charles Gannon

Contact: cgannon@ucmerced.edu

## Introduction

SubScript supports optional astropy unit integration. When enabled, all node properties returned by `nodedata()` are
astropy `Quantity` objects with physical units attached. By default, values are decomposed into the base unit system
`[Msun, Mpc, Myr]`, which matches the native Galacticus output units for most properties.

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as apu

from subscript.defaults import ParamKeys, Meta
from subscript.scripts.nodes import nodedata, nodecount
import subscript.scripts.nfilters as nf
from subscript.scripts.histograms import massfunction

gout = h5py.File('data/test.hdf5')

## Enabling Units

Use `enableUnitsFromGalacticus()` to read `unitsInSI` attributes from the HDF5 file and
globally enable unit-aware data access. After this call, all `nodedata()` results are
astropy Quantities.

In [ ]:
from subscript.units import enableUnitsFromGalacticus

enableUnitsFromGalacticus(gout)

print(f"Units enabled: {Meta.units_enable}")
print(f"Base units: {Meta.unit_bases}")

## Node Data with Units

Mass, distance, and time properties now carry their physical units.

In [ ]:
# Host halo mass - returned in solar masses
host_mass = nodedata(gout, key=ParamKeys.mass_basic, nfilter=nf.hosthalos)
print(f"Host halo mass: {host_mass[0]}")
print(f"Unit: {host_mass[0].unit}")

In [ ]:
# Virial radius - returned in Mpc
rvir = nodedata(gout, key=ParamKeys.rvir, nfilter=nf.hosthalos)
print(f"Host virial radius: {rvir[0]}")

# Convert to kpc
print(f"In kpc: {rvir[0].to(apu.kpc)}")

In [ ]:
# Time since last isolated - returned in Myr
time = nodedata(gout, key=ParamKeys.basic_time_last_isolated, nfilter=nf.hosthalos)
print(f"Time last isolated: {time[0]}")

# Convert to Gyr
print(f"In Gyr: {time[0].to(apu.Gyr)}")

## Unit Conversions

Astropy Quantities support seamless unit conversion with `.to()`.

In [ ]:
# Subhalo velocities: base units give Mpc/Myr, convert to km/s
vx = nodedata(gout, key=ParamKeys.satellite_velocity_x, nfilter=nf.subhalos)
print(f"Velocity (base units): {vx[0][:5]}")
print(f"Velocity (km/s): {vx[0][:5].to(apu.km / apu.s)}")

## Mass Function with Units

When units are enabled, histogram bins must be passed as Quantities so that astropy can
match the units of the data. Use `bins * apu.Msun` for mass bins.

In [ ]:
bins = np.logspace(9, 13, 30) * apu.Msun

out = massfunction(
    gout,
    bins=bins,
    key_mass=ParamKeys.mass_bound,
    nfilter=nf.subhalos,
    summarize=True,
    statfuncs=[np.mean, np.std],
)

dndm_mean, mbins = out[0]
dndm_std, _      = out[1]

fig, ax = plt.subplots()
ax.errorbar(mbins[:-1].value, dndm_mean.value, dndm_std.value)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(f'Mass [{mbins.unit}]')
ax.set_ylabel(f'dn/dM [{dndm_mean.unit}]')
ax.set_title('Subhalo Mass Function (mean +/- std over trees)')
plt.show()

## Extracting SI Conversion Factors

`galacticus_units_si()` reads the raw `unitsInSI` attribute from every dataset in the
HDF5 file and returns a dictionary of numerical conversion factors.

In [ ]:
from subscript.units import galacticus_units_si

conversion = galacticus_units_si(gout)

print(f"basicMass unitsInSI: {conversion[ParamKeys.mass_basic]:.6e} kg")
print(f"positionOrbitalX unitsInSI: {conversion[ParamKeys.x]:.6e} m")
print(f"satelliteVelocityX unitsInSI: {conversion[ParamKeys.satellite_velocity_x]:.1f} m/s")

## Disabling Units

To go back to raw numpy arrays, set `Meta.units_enable = False`.

In [ ]:
Meta.units_enable = False

mass_raw = nodedata(gout, key=ParamKeys.mass_basic, nfilter=nf.hosthalos)
print(f"Type: {type(mass_raw)}")
print(f"Value: {mass_raw[0]}")

In [ ]:
gout.close()